## Create Donut

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import data, img_as_float, color, exposure
from skimage.restoration import unwrap_phase
import sscPhantom
import numpy

def match_colorbar(ax):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    return make_axes_locatable(ax).append_axes("right", size="5%", pad=0.1)

def RemoveGrad_new( img, mask ):
    xy = numpy.argwhere( mask > 0)
    n = len(xy)
    y = xy[:,0].reshape([n,1])
    x = xy[:,1].reshape([n,1])
    F = numpy.array([ img[y[k],x[k]] for k in range(n) ]).reshape([n,1])
    mat = numpy.zeros([3,3])
    vec = numpy.zeros([3,1])
    mat[0,0] = (x*x).sum()
    mat[0,1] = (x*y).sum()
    mat[0,2] = (x).sum()
    mat[1,0] = mat[0,1]
    mat[1,1] = (y*y).sum()
    mat[1,2] = (y).sum()
    mat[2,0] = mat[0,2]
    mat[2,1] = mat[1,2]
    mat[2,2] = n
    vec[0,0] = (x*F).sum()
    vec[1,0] = (y*F).sum()
    vec[2,0] = (F).sum()
    eye = numpy.eye(mat.shape[0])
    eps = 1e-5 # valor tirado do *
    if 1: # com regularização
        abc = numpy.dot( numpy.linalg.inv(mat + eps * eye), vec).flatten() 
    else: # sem regularização
        abc = numpy.dot( numpy.linalg.inv(mat), vec).flatten()
    a = abc[0]
    b = abc[1]
    c = abc[2]
    new   = numpy.zeros(img.shape)
    row   = new.shape[0]
    col   = new.shape[1]
    XX,YY = numpy.meshgrid(numpy.arange(col),numpy.arange(row))
    new[y, x] = img[ y, x] - ( a*XX[y,x] + b*YY[y,x] + c )
    print('Fitted',a,b,c)
    #for k in range(n):
    #    new[y[k], x[k]] = img[ y[k], x[k]] - ( a*x[k] + b*y[k] + c )
    return new, a,b,c

N=256

# img = sscPhantom.mario.createMario(shape=N, noise=False, zoom=0.5)

params = { 'HowMany': 10,
           'radius': 0.07,
           'Rtorus': 0.5,
           'rtorus': 0.1}

nproc  = 16
energy = 7  #Kev
phantom1, phantom2 = sscPhantom.donuts.createDonuts( 128, nproc, energy, params )
phantom = phantom1 + phantom2

print('Type: ',type(phantom))

/home/ABTLUS/yuri.tonin/.local/lib/python3.6/site-packages/pkg_resources/__init__.py:119: PkgResourcesDeprecationWarning: 1.0.0. is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,


--> ssc-phantom: particles generated / elapsed: 43.36485028266907
sscPhantom: creating 2 128x128x128 shared arrays (Phantom)
Done: 128 images within 0.7463972568511963 sec
--> ssc-phantom: donuts created / elapsed: 0.7465329170227051
Type:  <class 'numpy.ndarray'>


## Get projections

In [ ]:
# Define probe

# Define probe positions

# Rotate phantom

# Get projection 

# Rescale frame (real and imag parts)



# Load an image as a floating-point grayscale
image0 = img_as_float(img)
image0 = exposure.rescale_intensity(image0, out_range=(0, 4 * np.pi))

# Wrap frame
""" Unwrap """
# x = np.linspace(0,image0.shape[0]-1,image0.shape[0])
# X, Y = np.meshgrid(x,x)
# a=0.1
# b=0.2
# c=2
# plane = a*X+b*Y +c
# image = image0 + plane

# Get ptycho projection "box"

# Get difpad

# Store difpad